In [1]:
# Updated plotting notebook
# Local imports
import argparse
import os
import sys
import time
import glob
import re

# Third party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
from datetime import datetime
import scipy.stats as stats

# Import cdo
from cdo import *
cdo = Cdo()

# Local imports
sys.path.append('/home/users/benhutch/skill-maps-historical')
import dictionaries as dic
import functions as fnc

In [2]:
# Set up the parameters for the first historical run
# psl year 2-9 DJFM global
variable = 'psl'
region = 'global'
region_grid = dic.gridspec_global
season = 'DJFM'
forecast_range = '2-9'
observations_path = dic.obs
obs_var_name = 'psl'
start_year = 1960
end_year = 2014
models = dic.psl_models


In [3]:
# Process the observations
# for year 2-9 DJFM global psl
obs = fnc.process_observations(variable, region, region_grid,
                                forecast_range, season, observations_path,
                                    obs_var_name)

File already exists
Regridded and selected region dataset: <xarray.DataArray 'time' (time: 259)>
array(['1959-01-01T00:00:00.000000000', '1959-02-01T00:00:00.000000000',
       '1959-03-01T00:00:00.000000000', ..., '2023-01-01T00:00:00.000000000',
       '2023-02-01T00:00:00.000000000', '2023-03-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1959-01-01 1959-02-01 ... 2023-03-01
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T
Forecast range: 2 - 9
Rolling mean range: 8


In [4]:
# Load the processed historical data
historical_data = fnc.load_processed_historical_data(dic.base_dir_historical,
                                                        models, variable,
                                                            region, forecast_range,
                                                                season)

processing model:  BCC-CSM2-MR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/BCC-CSM2-MR/global/years_2-9/DJFM/outputs/processed/*.nc
loading variant_label:  r1i1p1f1
loading variant_label:  r2i1p1f1
loading variant_label:  r3i1p1f1
processing model:  MPI-ESM1-2-HR
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/MPI-ESM1-2-HR/global/years_2-9/DJFM/outputs/processed/*.nc
loading variant_label:  r1i1p1f1
processing model:  CanESM5
files_path:  /home/users/benhutch/skill-maps-processed-data/historical/psl/CanESM5/global/years_2-9/DJFM/outputs/processed/*.nc
loading variant_label:  r10i1p1f1
loading variant_label:  r10i1p2f1


loading variant_label:  r11i1p1f1
loading variant_label:  r12i1p1f1
loading variant_label:  r13i1p1f1
loading variant_label:  r14i1p1f1
loading variant_label:  r15i1p1f1
loading variant_label:  r16i1p1f1
loading variant_label:  r17i1p1f1
loading variant_label:  r18i1p1f1
loading variant_label:  r19i1p1f1
loading variant_label:  r1i1p1f1
loading variant_label:  r1i1p2f1
loading variant_label:  r20i1p1f1
loading variant_label:  r21i1p1f1
loading variant_label:  r22i1p1f1
loading variant_label:  r23i1p1f1
loading variant_label:  r24i1p1f1
loading variant_label:  r25i1p1f1
loading variant_label:  r2i1p1f1
loading variant_label:  r2i1p2f1
loading variant_label:  r3i1p1f1
loading variant_label:  r3i1p2f1
loading variant_label:  r4i1p1f1
loading variant_label:  r4i1p2f1
loading variant_label:  r5i1p1f1
loading variant_label:  r5i1p2f1
loading variant_label:  r6i1p1f1
loading variant_label:  r6i1p2f1
loading variant_label:  r7i1p1f1
loading variant_label:  r7i1p2f1
loading variant_label:  r8i1

In [ ]:
# TODO: Test a function for processing the skill maps data
# Extract the variable data from the historical data
historical_data, historical_time = fnc.extract_historical_data(historical_data, 
                                                                variable)